In [ ]:
%aiida

In [ ]:
from aiida_cp2k.calculations import Cp2kCalculation

from aiida.orm import ArrayData
from aiida.engine import submit, run_get_node

from aiidalab_widgets_base import CodeDropdown, SubmitButtonWidget, StructureBrowserWidget
from aiidalab_widgets_base import ComputerDropdown

from apps.scanning_probe import analyze_structure
from apps.scanning_probe.viewer_details import ViewerDetails

import ase
import ase.io
import numpy as np
import nglview
from copy import deepcopy
from pprint import pprint

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

from aiida.common.exceptions import MissingEntryPointError 
try:
    Cp2kAFmWorkChain = WorkflowFactory('nanotech_empa.cp2k.afm') 
except MissingEntryPointError as e:
    print("Entry point not found. Did you perhaps forget to set up the plugins under 'Setup codes'?")
    raise e
    
from apps.scanning_probe import common
from apps.scanning_probe.metadata_widget import MetadataWidget

# Select structure

In [ ]:
atoms = None
slab_analyzed = None

def on_struct_change(c):
    global atoms, slab_analyzed
    structure = struct_browser.results.value
    if structure:
        atoms = structure.get_ase()
        atoms.pbc = [1, 1, 1]
        
        #slab_analyzed = find_mol.analyze_slab(atoms)
        viewer_widget.setup(atoms)
        
        if structure.creator is not None:
            text_calc_description.value = structure.creator.description

    
struct_browser = StructureBrowserWidget()
struct_browser.results.observe(on_struct_change, names='value')

viewer_widget = ViewerDetails()

display(ipw.VBox([struct_browser, viewer_widget]))

# Select computer and codes

In [ ]:
computer_drop = ComputerDropdown()

def on_computer_change(c):
    if computer_drop.selected_computer is not None:
        cp2k_codes = common.comp_plugin_codes(computer_drop.selected_computer.label, 'cp2k')
        afm_codes = common.comp_plugin_codes(computer_drop.selected_computer.label, 'spm.afm')

        # Clear as there seems to be a bug when label remains same,
        # then so does the value
        drop_cp2k.options = []
        drop_pp.options = []
        drop_2pp.options = []

        drop_cp2k.options = [(c.label, c) for c in cp2k_codes]
        drop_pp.options = [(c.label, c) for c in afm_codes if "_pp" in c.label]
        drop_2pp.options = [(c.label, c) for c in afm_codes if "_2pp" in c.label]
    
    
computer_drop._dropdown.observe(on_computer_change)

drop_cp2k = ipw.Dropdown(description="Cp2k code")

drop_pp = ipw.Dropdown(description="AFM PP")

drop_2pp = ipw.Dropdown(description="AFM 2PP")

on_computer_change(0)

elpa_check = ipw.Checkbox(
    value=True,
    description='use ELPA',
    disabled=False
)

display(computer_drop, drop_cp2k, elpa_check, drop_pp, drop_2pp)

# DFT PARAMETERS

In [ ]:
style = {'description_width': '80px'}
layout = {'width': '70%'}

def enable_spin(b):
    for w in [spin_up_text, spin_dw_text, vis_spin_button, multiplicity_text]:
        w.disabled = not uks_switch.value

def visualize_spin_guess(b):
    spin_up = [int(v)-1 for v in spin_up_text.value.split()]
    spin_dw = [int(v)-1 for v in spin_dw_text.value.split()]
    viewer_widget.reset()
    viewer_widget.highlight_atoms(spin_up, color='red', size=0.3, opacity=0.4)
    viewer_widget.highlight_atoms(spin_dw, color='blue', size=0.3, opacity=0.4)

uks_switch = ipw.ToggleButton(value=False,
                              description='Spin-polarized calculation',
                              style=style, layout={'width': '450px'})
uks_switch.observe(enable_spin, names='value')

spin_up_text = ipw.Text(placeholder='1 2 3',
                        description='Spin up',
                        disabled=True,
                        style=style, layout={'width': '370px'})
spin_dw_text = ipw.Text(placeholder='1 2 3',
                        description='Spin down',
                        disabled=True,
                        style=style, layout={'width': '370px'})
vis_spin_button = ipw.Button(description="Visualize",
                             disabled=True,
                             style = {'description_width': '0px'}, layout={'width': '75px'})
vis_spin_button.on_click(visualize_spin_guess)

multiplicity_text = ipw.IntText(value=1,
                           description='Multiplicity',
                           disabled=True,
                           style=style, layout={'width': '20%'})
sc_diag_check = ipw.Checkbox(
    value=False,
    description='self-consistent diagonalization',
    disabled=True
)
force_multiplicity_check = ipw.Checkbox(
    value=True,
    description='Fix multiplicity',
    disabled=True
)
protocol = ipw.Dropdown( 
            value="standard",
            options=[("Standard", "standard"), ("Low accuracy", "low_accuracy")],
            description="Protocol:",
            style={"description_width": "120px"},
        )
display(uks_switch, ipw.HBox([ipw.VBox([spin_up_text, spin_dw_text]), vis_spin_button]), 
ipw.HBox([multiplicity_text,sc_diag_check,force_multiplicity_check]),protocol)

In [ ]:
def enable_smearing(b):
    temperature_text.disabled = not smear_switch.value

smear_switch = ipw.ToggleButton(value=False,
                              description='Enable Fermi-Dirac smearing',
                              style=style, layout={'width': '450px'})
smear_switch.observe(enable_smearing, names='value')

temperature_text = ipw.FloatText(value=300.0,
                           description='Temperature [K]',
                           disabled=True,
                           style={'description_width': '100px'}, layout={'width': '20%'})


display(smear_switch, temperature_text)

# AFM parameters

In [ ]:
style = {'description_width': '140px'}
layout = {'width': '70%'}
layout_small = {'width': '30%'}

scanstep_floattxt = ipw.BoundedFloatText(
                        description='Scan dx (ang)',
                        min=0.05,
                        max=0.5,
                        step=0.05,
                        value=0.1,
                        style=style, layout=layout_small)

scanminz_floattxt = ipw.BoundedFloatText(
                        description='Scan min z (ang)',
                        min=0.0,
                        max=5.0,
                        step=0.1,
                        value=3.5,
                        style=style, layout=layout_small)

scanmaxz_floattxt = ipw.BoundedFloatText(
                        description='Scan max z (ang)',
                        min=5.0,
                        max=10.0,
                        step=0.1,
                        value=8.5,
                        style=style, layout=layout_small)

amp_floattxt = ipw.FloatText(
                        description='Amplitude (ang)',
                        step=0.1,
                        value=1.4,
                        style=style, layout=layout_small)

f0_cantilever_floattxt = ipw.FloatText(
                        description='Cantilever f0',
                        step=0.1,
                        value=22352.5,
                        style=style, layout=layout_small)


display(scanstep_floattxt, scanminz_floattxt, scanmaxz_floattxt, amp_floattxt, f0_cantilever_floattxt)

In [ ]:
drop_2pp_resp = ipw.Dropdown(description="2PP RESP model",
                            style=style, layout=layout_small,
                            options = { # ChargeCuUp, ChargeCuDown, Ccharge, Ocharge
                                'pentacene': [-0.0669933, -0.0627402, 0.212718, -0.11767],
                                'ptcda':     [     -0.05,      -0.07,     0.23,    -0.13]
                            })

display(drop_2pp_resp)

In [ ]:
def create_pp_parameterdata(ase_geom):
    cell = ase_geom.cell
    top_z = np.max(ase_geom.positions[:, 2])
    dx = scanstep_floattxt.value
    paramdata = Dict(dict={
        'probeType':    'Otip',
        'charge':       -0.028108681223969645,
        'sigma':        0.7,
        'tip':          's',
        'klat':         0.34901278868090491,
        'krad':         21.913190531846034,
        'r0Probe':      [0.0, 0.0, 2.97],
        'PBC':          'False',
        'gridA':        list(cell[0]),
        'gridB':        list(cell[1]),
        'gridC':        list(cell[2]),
        'scanMin':      [0.0, 0.0, np.round(top_z, 1)+scanminz_floattxt.value],
        'scanMax':      [cell[0,0], cell[1,1], np.round(top_z, 1)+scanmaxz_floattxt.value],
        'scanStep':     [dx, dx, dx],
        'Amplitude':    amp_floattxt.value,
        'f0Cantilever': f0_cantilever_floattxt.value
    })
    return paramdata

def create_2pp_parameterdata(ase_geom):
    cell = ase_geom.cell
    top_z = np.max(ase_geom.positions[:, 2])
    dx = scanstep_floattxt.value
    resp = drop_2pp_resp.value
    paramdata = Dict(dict={
        'Catom':        'Ctip',
        'Oatom':        'Otip',
        'ChargeCuUp':   resp[0],
        'ChargeCuDown': resp[1],
        'Ccharge':      resp[2],
        'Ocharge':      resp[3],
        'sigma':        0.7,
        'Cklat':        0.24600212465950813,
        'Oklat':        0.15085476515590224,
        'Ckrad':        20,
        'Okrad':        20,
        'rC0':          [0.0, 0.0, 1.82806112489999961213],
        'rO0':          [0.0, 0.0, 1.14881347770000097341],
        'PBC':          'False',
        'gridA':        list(cell[0]),
        'gridB':        list(cell[1]),
        'gridC':        list(cell[2]),
        'scanMin':      [0.0, 0.0, np.round(top_z, 1)+scanminz_floattxt.value],
        'scanMax':      [cell[0,0], cell[1,1], np.round(top_z, 1)+scanmaxz_floattxt.value],
        'scanStep':     [dx, dx, dx],
        'Amplitude':    amp_floattxt.value,
        'f0Cantilever': f0_cantilever_floattxt.value,
        'tip':          'None',
        'Omultipole':   's',
    })
    return paramdata

# Submission

In [ ]:
# Description for the calculation (try to read from the structure creator)

text_calc_description = ipw.Text(description='Description:', layout={'width': '45%'})
display(text_calc_description)

In [ ]:
def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not computer_drop.selected_computer:
            print("Please select a computer.")
            return
        if not drop_cp2k.value or not drop_pp.value or not drop_2pp.value:
            print("Please select all the codes.")
            return

        dft_params_dict = {
            'elpa_switch':     elpa_check.value,
            'sc_diag':           sc_diag_check.value,
            'protocol':          protocol.value,
            'force_multiplicity':          force_multiplicity_check.value,
            'periodic':         'XYZ',
            'uks':             uks_switch.value,
        }
        if uks_switch.value:
            dft_params_dict['spin_up_guess'] = [int(v)-1 for v in spin_up_text.value.split()]
            dft_params_dict['spin_dw_guess'] = [int(v)-1 for v in spin_dw_text.value.split()]
            dft_params_dict['multiplicity']  = multiplicity_text.value
            dft_params_dict['force_multiplicity']  = force_multiplicity_check.value
            
        if smear_switch.value:
            dft_params_dict['smear_t'] = temperature_text.value            
        
        dft_params = Dict(dict=dft_params_dict)            
        
        struct = struct_browser.results.value
        
        ase_geom = struct.get_ase()
        cell_array = ArrayData()
        cell_array.set_array('cell', np.diag(ase_geom.cell))

        afm_pp_params = create_pp_parameterdata(ase_geom)
        afm_2pp_params = create_2pp_parameterdata(ase_geom)
        
        ## Try to access the restart-wfn file ##
        selected_comp = drop_cp2k.value.computer
        try:
            wfn_file_path = common.find_struct_wf(struct, selected_comp)
        except:
            wfn_file_path = ""
        if wfn_file_path == "":
            print("Info: didn't find any accessible .wfn file.")
            
        node = submit(
            Cp2kAfmWorkChain,
            cp2k_code=drop_cp2k.value,
            structure=struct,
            wfn_file_path=Str(wfn_file_path),
            dft_params=dft_params,
            afm_pp_code=drop_pp.value,
            afm_pp_params=afm_pp_params,
            afm_2pp_code=drop_2pp.value,
            afm_2pp_params=afm_2pp_params,
            metadata={'description': text_calc_description.value}
        )
                
        # set calculation version; also used to determine post-processing
        node.set_extra("version", 0)
        
        print()
        print("Submitted:")
        print(node)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)